In [192]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [193]:
rawfact = pd.read_csv(r'C:\Users\jkaru\powerbi-capstone-Automotive-parts-02\Data\Raw\fact_table.csv')

In [194]:
rawequipment = pd.read_csv(r'C:\Users\jkaru\powerbi-capstone-Automotive-parts-02\Data\Raw\equipment_dimension.csv')

In [195]:
rawlocation = pd.read_csv(r'C:\Users\jkaru\powerbi-capstone-Automotive-parts-02\Data\Raw\location_dimension.csv')

In [196]:

rawsensor = pd.read_csv(r'C:\Users\jkaru\powerbi-capstone-Automotive-parts-02\Data\Raw\sensor_dimension.csv')
rawtime = pd.read_csv(r'C:\Users\jkaru\powerbi-capstone-Automotive-parts-02\Data\Raw\time_dimension.csv')

In [197]:
#check for missing values in the fact table
missing_values = rawfact.isnull().sum()
#print the missing values
print("Missing values in the fact table:", missing_values)

Missing values in the fact table: SensorDataID    0
Timestamp       0
EquipmentID     0
TimeID          0
SensorID        0
LocationID      0
Temperature     0
Vibration       0
Pressure        0
Voltage         0
dtype: int64


### Checking and changing datatypes to the appropriate format

In [198]:
#checking the data types of the columns in the raw fact table
rawfact.dtypes


SensorDataID     object
Timestamp        object
EquipmentID      object
TimeID            int64
SensorID         object
LocationID       object
Temperature     float64
Vibration       float64
Pressure        float64
Voltage         float64
dtype: object

In [199]:
#Change the data types of the columns in the raw fact table
rawfact["EquipmentID"] = rawfact["EquipmentID"].astype(str)
rawfact["LocationID"] = rawfact["LocationID"].astype(str)
rawfact["SensorID"] = rawfact["SensorID"].astype(str)
rawfact["TimeID"] = rawfact["TimeID"].astype(str)
rawfact["Temperature"] = rawfact["Temperature"].astype(float)
rawfact["Vibration"] = rawfact["Vibration"].astype(float)
rawfact["Pressure"] = rawfact["Pressure"].astype(float)
rawfact["Voltage"] = rawfact["Voltage"].astype(float)
rawfact["Timestamp"] = pd.to_datetime(rawfact["Timestamp"], format='%Y-%m-%d')


In [200]:
#checking the data types of the columns in the raw fact table again
rawfact.dtypes

SensorDataID            object
Timestamp       datetime64[ns]
EquipmentID             object
TimeID                  object
SensorID                object
LocationID              object
Temperature            float64
Vibration              float64
Pressure               float64
Voltage                float64
dtype: object

## Equipment Health Scoring System

In [201]:
#check for skewness in float columns of the fact table individually starting with Temperature
temperature_skewness = rawfact['Temperature'].skew()
print("Skewness in Temperature column:", temperature_skewness)
#check for skewness in Vibration column
vibration_skewness = rawfact['Vibration'].skew()
print("Skewness in Vibration column:", vibration_skewness)
#check for skewness in Pressure column
pressure_skewness = rawfact['Pressure'].skew()
print("Skewness in Pressure column:", pressure_skewness)
#check for skewness in Voltage column
voltage_skewness = rawfact['Voltage'].skew()
print("Skewness in Voltage column:", voltage_skewness)




Skewness in Temperature column: -0.0382253947268469
Skewness in Vibration column: 0.18554841804653366
Skewness in Pressure column: -0.02631419765206785
Skewness in Voltage column: -0.0898408422815011


In [202]:
# Add Calculated Fields
# Normalize: higher is better → lower values are healthier
rawfact["TempNorm"] = (rawfact["Temperature"].max() - rawfact["Temperature"]) / (rawfact["Temperature"].max() - rawfact["Temperature"].min())
rawfact["VibNorm"] = (rawfact["Vibration"].max() - rawfact["Vibration"]) / (rawfact["Vibration"].max() - rawfact["Vibration"].min())
rawfact["VoltNorm"] = (rawfact["Voltage"].max() - rawfact["Voltage"]) / (rawfact["Voltage"].max() - rawfact["Voltage"].min())
rawfact["PresNorm"] = (rawfact["Pressure"].max() - rawfact["Pressure"]) / (rawfact["Pressure"].max() - rawfact["Pressure"].min())

# Clip to [0, 1] in case of rounding issues
for col in ["TempNorm", "VibNorm", "VoltNorm", "PresNorm"]:
    rawfact[col] = rawfact[col].clip(0, 1)

# Weights must add up to 1.0
temp_w = 0.3
vib_w = 0.3
volt_w = 0.2
pres_w = 0.2

# Compute HealthScore
rawfact["HealthScore"] = 100 * (
    temp_w * rawfact["TempNorm"] +
    vib_w * rawfact["VibNorm"] +
    volt_w * rawfact["VoltNorm"] +
    pres_w * rawfact["PresNorm"]
).round(4)


## Parameterised Data Pipelines for: 
-  Maintenance Avoidance Cost Calculator
- Data Validation

In [203]:
import numpy as np
import pandas as pd

def simulate_maintenance_costs(
    rawfact,
    failure_threshold=50,
    reactive_cost_range=(2000, 5000),
    preventive_cost_range=(300, 1200)
):
    """
    Adds cost-related columns to a fact table based on a HealthScore and failure prediction.

    Parameters:
    - rawfact (pd.DataFrame): your fact table with 'HealthScore' and 'PredictedFailure'
    - failure_threshold (int): cutoff value for flagging failure
    - reactive_cost_range (tuple): min/max cost for reactive maintenance
    - preventive_cost_range (tuple): min/max cost for preventive maintenance

    Returns:
    - rawfact with new columns: ReactiveCost, PreventiveCost, CostAvoided
    """
    
    # Ensure 'PredictedFailure' is based on threshold
    rawfact["PredictedFailure"] = rawfact["HealthScore"] < failure_threshold

    # Simulate costs based on failure status
    rawfact["ReactiveCost"] = np.where(
        rawfact["PredictedFailure"],
        np.random.randint(reactive_cost_range[0], reactive_cost_range[1], size=len(rawfact)),
        0
    )
    
    rawfact["PreventiveCost"] = np.where(
        rawfact["PredictedFailure"],
        np.random.randint(preventive_cost_range[0], preventive_cost_range[1], size=len(rawfact)),
        0
    )
    
    rawfact["CostAvoided"] = rawfact["ReactiveCost"] - rawfact["PreventiveCost"]
    
    return rawfact
rawfact=simulate_maintenance_costs(rawfact)



rawfact


,SensorDataID,Timestamp,EquipmentID,TimeID,SensorID,LocationID,Temperature,Vibration,Pressure,Voltage,TempNorm,VibNorm,VoltNorm,PresNorm,HealthScore,PredictedFailure,ReactiveCost,PreventiveCost,CostAvoided
0,SD_0001,2024-05-08,EQT_008,109,SNS_006,LOC_02,94.374549,1.039699,1.426055,247.027209,0.157070,0.849858,0.226750,0.646276,47.67,True,3703,612,3091
1,SD_0002,2024-02-27,EQT_018,152,SNS_003,LOC_05,73.439602,3.996542,2.380632,200.089409,0.491481,0.641585,0.666810,0.331937,53.97,False,0,0,0
2,SD_0003,2024-05-01,EQT_012,17,SNS_006,LOC_05,60.423923,4.856926,2.429243,177.712353,0.699390,0.580982,0.876604,0.315929,62.26,False,0,0,0
3,SD_0004,2024-01-01,EQT_015,133,SNS_010,LOC_06,77.710378,3.713351,2.417804,200.356708,0.423260,0.661532,0.664304,0.319696,52.22,False,0,0,0
4,SD_0005,2024-08-26,EQT_009,148,SNS_010,LOC_04,68.309954,3.456910,1.911094,236.956233,0.573420,0.679596,0.321169,0.486554,53.74,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,SD_0996,2024-09-20,EQT_005,189,SNS_004,LOC_04,90.653727,7.907428,2.571239,217.593264,0.216506,0.366112,0.502705,0.269171,32.92,True,4230,591,3639
996,SD_0997,2024-05-28,EQT_008,287,SNS_004,LOC_05,75.534175,5.114377,2.021140,234.527745,0.458022,0.562848,0.343937,0.450316,46.51,True,2136,753,1383
997,SD_0998,2024-03-21,EQT_013,208,SNS_009,LOC_05,61.865109,5.912940,1.173630,214.501836,0.676369,0.506599,0.531688,0.729398,60.71,False,0,0,0
998,SD_0999,2024-08-02,EQT_008,114,SNS_001,LOC_05,73.038119,5.486788,1.769184,221.620366,0.497894,0.536616,0.464949,0.533285,51.00,False,0,0,0


In [204]:

def validate_rawfact(rawfact):
    """
    Runs data validation checks on the rawfact table.

    Checks:
    1. HealthScore is between 0 and 100
    2. No nulls in key columns (SensorID, EquipmentID, TimeID)
    3. Timestamp format is valid

    Raises:
        AssertionError if any check fails
    Returns:
        True if all validations pass
    """
    # 1. HealthScore must be within 0–100
    assert rawfact['HealthScore'].between(0, 100).all(), "❌ HealthScore out of range (0–100)"

    # 2. Foreign key columns must not have nulls
    key_cols = ['SensorID', 'EquipmentID', 'TimeID']
    assert not rawfact[key_cols].isnull().any().any(), f"❌ Missing values in key columns: {key_cols}"

    # 3. Timestamp must be valid datetime
    assert pd.to_datetime(rawfact['Timestamp'], errors='coerce').notna().all(), "❌ Invalid timestamps in 'Timestamp' column"

    print("✅ Data validation passed!")
    return True




In [205]:
rawfact

,SensorDataID,Timestamp,EquipmentID,TimeID,SensorID,LocationID,Temperature,Vibration,Pressure,Voltage,TempNorm,VibNorm,VoltNorm,PresNorm,HealthScore,PredictedFailure,ReactiveCost,PreventiveCost,CostAvoided
0,SD_0001,2024-05-08,EQT_008,109,SNS_006,LOC_02,94.374549,1.039699,1.426055,247.027209,0.157070,0.849858,0.226750,0.646276,47.67,True,3703,612,3091
1,SD_0002,2024-02-27,EQT_018,152,SNS_003,LOC_05,73.439602,3.996542,2.380632,200.089409,0.491481,0.641585,0.666810,0.331937,53.97,False,0,0,0
2,SD_0003,2024-05-01,EQT_012,17,SNS_006,LOC_05,60.423923,4.856926,2.429243,177.712353,0.699390,0.580982,0.876604,0.315929,62.26,False,0,0,0
3,SD_0004,2024-01-01,EQT_015,133,SNS_010,LOC_06,77.710378,3.713351,2.417804,200.356708,0.423260,0.661532,0.664304,0.319696,52.22,False,0,0,0
4,SD_0005,2024-08-26,EQT_009,148,SNS_010,LOC_04,68.309954,3.456910,1.911094,236.956233,0.573420,0.679596,0.321169,0.486554,53.74,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,SD_0996,2024-09-20,EQT_005,189,SNS_004,LOC_04,90.653727,7.907428,2.571239,217.593264,0.216506,0.366112,0.502705,0.269171,32.92,True,4230,591,3639
996,SD_0997,2024-05-28,EQT_008,287,SNS_004,LOC_05,75.534175,5.114377,2.021140,234.527745,0.458022,0.562848,0.343937,0.450316,46.51,True,2136,753,1383
997,SD_0998,2024-03-21,EQT_013,208,SNS_009,LOC_05,61.865109,5.912940,1.173630,214.501836,0.676369,0.506599,0.531688,0.729398,60.71,False,0,0,0
998,SD_0999,2024-08-02,EQT_008,114,SNS_001,LOC_05,73.038119,5.486788,1.769184,221.620366,0.497894,0.536616,0.464949,0.533285,51.00,False,0,0,0


In [206]:
rawfact.to_csv(r'C:\Users\jkaru\powerbi-capstone-Automotive-parts-02\Data\cleaned\fact_table_cleaned.csv', index=False)


In [207]:
#save the clean data to cleaned folder

